In [1]:
from pathlib import Path
from utils import read_input
from collections import Counter, deque
import numpy as np
from bresenham import bresenham
from statistics import mean, median, mode
from itertools import combinations, product, pairwise, permutations
from tqdm import tqdm
import pandas as pd
from rich import print
import matplotlib.pyplot as plt

In [2]:
test = Path("input/day16/test.txt")
data = Path("input/day16/data.txt")

In [3]:
def hex_to_binary(inputstr):
    as_int = int(inputstr, base=16)
    as_bin_str = str(bin(as_int))[2:]
    h_size = len(inputstr) * 4
    return as_bin_str.zfill(h_size)

def binary_to_decimal(inputstr):
    return int(inputstr, base=2)

def parse_header(inputbinary):
    packet_version = binary_to_decimal(inputbinary[:3])
    packet_type = binary_to_decimal(inputbinary[3:6])
    return packet_version, packet_type, inputbinary[6:]

def convert_literal_value(inputbinary):
    i = 0
    cumul_num = ""
    while True:
        group = inputbinary[i: i + 5]
        number_part = group[1:]
        cumul_num = cumul_num + number_part
        if group[0] == "0":
            remainder = inputbinary[i + 5:]
            break
        i += 5
    
    return binary_to_decimal(cumul_num), remainder
    

In [4]:
test = hex_to_binary("D2FE28")
v, t, rest = parse_header(test)
convert_literal_value(rest)

(2021, '000')

In [5]:
parse_header(test)

(6, 4, '101111111000101000')

In [34]:
def parse_operator_packet_0(packet, versions):
    length_of_subpackets_in_bits = binary_to_decimal(packet[0:15])
    subpackets = packet[15:15 + length_of_subpackets_in_bits]
    remainder = packet[15 + length_of_subpackets_in_bits:]
    assert len(subpackets) == length_of_subpackets_in_bits
    while subpackets:
        version, type_id, rest = parse_header(subpackets)
        versions.append(version)
        if type_id == 4:
            parsed, subpackets = parse_literal_value(rest)
            print(binary_to_decimal(parsed))
        else:
            operator = rest[0]
            if operator == "0":
                versions, _ = parse_operator_packet_0(rest[1:], versions)
            else:
                versions, _ = parse_operator_packet_1(rest[1:], versions)
    return versions, remainder
    
    
def parse_operator_packet_1(packet, versions):
    number_of_subpackets = binary_to_decimal(packet[0:11])
    subpackets = packet[11:]
    
    subpackets_parsed = 0
    while subpackets_parsed < number_of_subpackets: 
        version, type_id, rest = parse_header(subpackets)
        versions.append(version)
        if type_id == 4:
            parsed, subpackets = parse_literal_value(rest)
#             print(binary_to_decimal(parsed))
        else:
            operator = rest[0]
            if operator == "0":
                versions, _ = parse_operator_packet_0(rest[1:], versions)
            else:
                versions, _ = parse_operator_packet_1(rest[1:], versions)
        subpackets_parsed += 1
    return versions, ''
            

def parse_literal_value(inputbinary):
    i = 0
    cumul_num = ""
    while True:
        group = inputbinary[i: i + 5]
        number_part = group[1:]
        cumul_num = cumul_num + number_part
        if group[0] == "0":
            remainder = inputbinary[i + 5:]
            break
        i += 5
    return cumul_num, remainder


def parse(inputbinary, versions=None):
    versions = versions or []
    version, type_id, rest = parse_header(inputbinary)
    versions.append(version)
    if type_id == 4:
        return versions
    else:
        operator = rest[0]
        if operator == "0":
            return parse_operator_packet_0(rest[1:], versions)
        else:
            return parse_operator_packet_1(rest[1:], versions)

        
def parse1(inputbinary, versions=None):
    versions = versions or []
    version, type_id, packet_contents = parse_header(inputbinary)
    if type_id == 4:
        # literal value
        versions.append(version)
        return versions
    else:
        # operator
        print(version, type_id, packet_contents)
        operator_length_type = packet_contents[0]
        packet_contents = packet_contents[1:]
        if operator_length_type == "0":
            if len(packet_contents) < 16:  # we need at least 16 bits, 15 for the length of the packet
                return versions
            versions.append(version)
            length_of_subpackets_in_bits = binary_to_decimal(packet_contents[0:15])
            packet_contents = packet_contents[15:]
            versions = parse1(packet_contents[0:length_of_subpackets_in_bits], versions)
            remaining_packet = packet_contents[length_of_subpackets_in_bits:]
            return parse1(remaining_packet, versions)
        else:
            if len(packet_contents) < 18:  # we need at least 18 bits, 11 for the number of packets, 7 for the headers and value of the first subpacket 
                return versions
            versions.append(version)
            number_of_subpackets = binary_to_decimal(packet_contents[0:11])
            packet_contents = packet_contents[11:]
            
            print('pending')
        
            


In [38]:
test = hex_to_binary("A0016C880162017C3686B18A3D4780")
versions = parse1(test)
print(versions)
sum(versions)

5 0 00000000010110110010001000000000010110001000000001011111000011011010000110101100011000101
0001111010100011110000000

1 0 1000000000010110001000000001011111000011011010000110101100011000101000111101010001111

pending

0 0 0

[]

0

In [69]:
versions1

[4, 1, 5, 6]

In [146]:
parse_header(test)

(1, 6, '00000000000110111101000101001010010001001000000000')

In [147]:
_146[2] == '00000000000110111101000101001010010001001000000000'

True

In [162]:
version, type_id, rest = parse_header(test)

In [163]:
operator = rest[0]
rest = rest[1:]

In [164]:
num_of_bits = rest[:15]
rest = rest[15:]
subpacket_bits = binary_to_decimal(num_of_bits)
rest = rest[:27]

In [165]:
rest

'110100010100101001000100100'

In [166]:
len(rest)

27

In [168]:
parse_header(rest)

(6, 4, '010100101001000100100')

In [169]:
parse_literal_value('010100101001000100100')

010100101001000100100

('1010', '0101001000100100')

In [170]:
parse_header('0101001000100100')

(2, 4, '1000100100')

In [171]:
parse_literal_value('1000100100')

1000100100

('00010100', '')

In [175]:
binary_to_decimal('1000100100')

548